In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np

In [2]:
def compare(calc1, calc2, syr, eyr):
    """
    Compare aggregate tax liability under two calculators
    where calc1 is the baseline and calc2 is the new/reform
    """
    # dictionary to hold aggregate liability
    tax_liability = {'year': [], 'base': [], 'new': [], 'diff': []}

    # loop through each year collecting aggregate tax liabilities
    for year in range(syr, eyr + 1):
        try:
            calc1.advance_to_year(year)
            calc1.calc_all()
            tax1 = calc1.weighted_total('combined')
        except ValueError:
            tax1 = np.nan
        calc2.advance_to_year(year)
        calc2.calc_all()
        tax2 = calc2.weighted_total('combined')
        diff = tax2 - tax1
        tax_liability['year'].append(year)
        tax_liability['base'].append(tax1)
        tax_liability['new'].append(tax2)
        tax_liability['diff'].append(diff)
    return tax_liability

# PUF

In [3]:
puf_path = 'puf_data/puf.csv'
gfactors_path = 'puf_stage1/growfactors.csv'
puf_weights = pd.read_csv('puf_stage2/puf_weights.csv.gz')
puf_ratios = pd.read_csv('puf_stage3/puf_ratios.csv', index_col=0).transpose()
puf_ratios_path = 'puf_stage3/puf_ratios.csv'

In [4]:
calc_base = tc.Calculator(records=tc.Records(puf_path), policy=tc.Policy())
gfactors_new = tc.GrowFactors(gfactors_path)
recs_new = tc.Records(puf_path, gfactors=gfactors_new,
                      weights=puf_weights,
                      adjust_ratios=puf_ratios_path)
# adjustment ratios aren't being read correctly for some reason
setattr(recs_new, 'ADJ', puf_ratios)
pol_new = tc.Policy(gfactors=gfactors_new, num_years=16)
puf_consumption = tc.Consumption(num_years=16)
puf_behavior = tc.Behavior(num_years=16)
calc_new = tc.Calculator(records=recs_new, policy=pol_new,
                         consumption=puf_consumption,
                         behavior=puf_behavior)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [5]:
puf_comps = compare(calc_base, calc_new, 2018, 2028)

In [6]:
puf_comp_df = pd.DataFrame(puf_comps)
puf_comp_df.style.format("${:,.2f}", subset=['base', 'new', 'diff'])

,year,base,new,diff
0,2018,"$2,698,886,061,445.21","$2,698,886,061,613.82",$168.61
1,2019,"$2,807,971,902,584.51","$2,807,971,902,733.95",$149.44
2,2020,"$2,914,941,958,015.72","$2,914,941,958,325.36",$309.65
3,2021,"$3,032,125,291,050.99","$3,032,125,291,513.66",$462.67
4,2022,"$3,161,917,824,343.74","$3,161,917,824,183.44",$-160.29
5,2023,"$3,304,099,583,484.29","$3,304,099,583,922.09",$437.80
6,2024,"$3,453,242,757,532.03","$3,453,242,757,907.35",$375.32
7,2025,"$3,608,930,900,218.76","$3,608,930,939,498.62","$39,279.85"
8,2026,"$4,005,258,346,192.27","$4,005,258,344,747.31","$-1,444.96"
9,2027,"$4,185,552,553,605.42","$4,185,552,551,884.73","$-1,720.69"


# CPS

In [7]:
cps_path = 'cps_data/cps.csv.gz'
cps_weights = pd.read_csv('cps_stage2/cps_weights.csv.gz')
gfactors_cps = tc.GrowFactors(gfactors_path)

In [8]:
cps_base = tc.Calculator(records=tc.Records.cps_constructor(), policy=tc.Policy())
cps_recs_new = tc.Records(cps_path, gfactors=gfactors_new,
                          weights=cps_weights,
                          adjust_ratios=None,
                          start_year=2014)
cps_pol_new = tc.Policy(gfactors=gfactors_new, num_years=16)
cps_consumption = tc.Consumption(num_years=16)
cps_behavior = tc.Behavior(num_years=16)
cps_new = tc.Calculator(records=cps_recs_new, policy=cps_pol_new,
                        consumption=cps_consumption,
                        behavior=cps_behavior)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [9]:
cps_comps = compare(cps_base, cps_new, 2018, 2028)

In [10]:
cps_comps_df = pd.DataFrame(cps_comps)
cps_comps_df.style.format("${:,.2f}", subset=['base', 'new', 'diff'])

,year,base,new,diff
0,2018,"$2,487,514,451,921.76","$2,487,514,451,921.76",$0.00
1,2019,"$2,611,763,363,898.20","$2,611,763,363,898.20",$0.00
2,2020,"$2,730,033,510,508.89","$2,730,033,510,508.89",$0.00
3,2021,"$2,857,078,503,025.49","$2,857,078,503,025.49",$0.00
4,2022,"$2,992,118,422,141.43","$2,992,118,422,141.43",$0.00
5,2023,"$3,133,292,206,467.64","$3,133,292,206,467.64",$0.00
6,2024,"$3,277,055,269,301.72","$3,277,055,269,301.72",$0.00
7,2025,"$3,420,333,632,991.05","$3,420,333,679,132.29","$46,141.24"
8,2026,"$3,775,740,844,507.71","$3,775,740,844,507.71",$0.00
9,2027,"$3,938,858,730,194.02","$3,938,858,730,194.02",$0.00
